In [39]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
import xgboost as xgb

### https://www.kaggle.com/datasets/shebrahimi/financial-distress

In [40]:
data = pd.read_csv('/kaggle/input/financial-distress/Financial Distress.csv')

In [41]:
data

,Company,Time,Financial Distress,x1,x2,x3,x4,x5,x6,x7,...,x74,x75,x76,x77,x78,x79,x80,x81,x82,x83
0,1,1,0.010636,1.2810,0.022934,0.87454,1.21640,0.060940,0.188270,0.52510,...,85.437,27.07,26.102,16.000,16.0,0.2,22,0.060390,30,49
1,1,2,-0.455970,1.2700,0.006454,0.82067,1.00490,-0.014080,0.181040,0.62288,...,107.090,31.31,30.194,17.000,16.0,0.4,22,0.010636,31,50
2,1,3,-0.325390,1.0529,-0.059379,0.92242,0.72926,0.020476,0.044865,0.43292,...,120.870,36.07,35.273,17.000,15.0,-0.2,22,-0.455970,32,51
3,1,4,-0.566570,1.1131,-0.015229,0.85888,0.80974,0.076037,0.091033,0.67546,...,54.806,39.80,38.377,17.167,16.0,5.6,22,-0.325390,33,52
4,2,1,1.357300,1.0623,0.107020,0.81460,0.83593,0.199960,0.047800,0.74200,...,85.437,27.07,26.102,16.000,16.0,0.2,29,1.251000,7,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3667,422,10,0.438020,2.2605,0.202890,0.16037,0.18588,0.175970,0.198400,2.22360,...,100.000,100.00,100.000,17.125,14.5,-7.0,37,0.436380,4,41
3668,422,11,0.482410,1.9615,0.216440,0.20095,0.21642,0.203590,0.189870,1.93820,...,91.500,130.50,132.400,20.000,14.5,-16.0,37,0.438020,5,42
3669,422,12,0.500770,1.7099,0.207970,0.26136,0.21399,0.193670,0.183890,1.68980,...,87.100,175.90,178.100,20.000,14.5,-20.2,37,0.482410,6,43
3670,422,13,0.611030,1.5590,0.185450,0.30728,0.19307,0.172140,0.170680,1.53890,...,92.900,203.20,204.500,22.000,22.0,6.4,37,0.500770,7,44


In [42]:
data.isnull().sum().sum()

0

In [43]:
data = data.drop(['Company','Time'],axis=1)

In [44]:
dummie = pd.get_dummies(data['x81'],prefix='x81').astype(np.float)
data = pd.concat([data,dummie],axis=1)
data = data.drop('x81',axis=1)

/tmp/ipykernel_32/4281268265.py:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dummie = pd.get_dummies(data['x81'],prefix='x81').astype(np.float)


In [45]:
y = data['Financial Distress']
x = data.drop('Financial Distress',axis=1)

In [46]:
scaler = StandardScaler()
x = scaler.fit_transform(x)

In [48]:
x_train , x_test , y_train , y_test = train_test_split(x,y,train_size=0.7,random_state=13)
x_train , x_val , y_train , y_val = train_test_split(x_train,y_train,test_size=0.2,random_state=13)

In [49]:
linear = LinearRegression()
linear.fit(x_train , y_train)
linear.score(x_test,y_test)

-0.6386887358332225

In [50]:
params = {'learning_rate':0.01,'max_depth':6,'lambda':0.01}

In [51]:
trainmatrix = xgb.DMatrix(x_train , label = y_train)
valmatrix = xgb.DMatrix(x_val , label = y_val)
testmatrix = xgb.DMatrix(x_test , label = y_test)

In [52]:
model = xgb.train(
    params
    ,trainmatrix
    ,num_boost_round=10000
    ,evals=[(valmatrix,'eval')]
    ,early_stopping_rounds=10
)

[0]	eval-rmse:5.86607
[1]	eval-rmse:5.86292
[2]	eval-rmse:5.85983
[3]	eval-rmse:5.85683
[4]	eval-rmse:5.85389
[5]	eval-rmse:5.85104
[6]	eval-rmse:5.84825
[7]	eval-rmse:5.84554
[8]	eval-rmse:5.84289
[9]	eval-rmse:5.84032
[10]	eval-rmse:5.83780
[11]	eval-rmse:5.83533
[12]	eval-rmse:5.83292
[13]	eval-rmse:5.83059
[14]	eval-rmse:5.82826
[15]	eval-rmse:5.82604
[16]	eval-rmse:5.82383
[17]	eval-rmse:5.82172
[18]	eval-rmse:5.81965
[19]	eval-rmse:5.81760
[20]	eval-rmse:5.81564
[21]	eval-rmse:5.81369
[22]	eval-rmse:5.81164
[23]	eval-rmse:5.80978
[24]	eval-rmse:5.80781
[25]	eval-rmse:5.80584
[26]	eval-rmse:5.80411
[27]	eval-rmse:5.80221
[28]	eval-rmse:5.80034
[29]	eval-rmse:5.79866
[30]	eval-rmse:5.79685
[31]	eval-rmse:5.79507
[32]	eval-rmse:5.79344
[33]	eval-rmse:5.79173
[34]	eval-rmse:5.79001
[35]	eval-rmse:5.78854
[36]	eval-rmse:5.78711
[37]	eval-rmse:5.78579
[38]	eval-rmse:5.78436
[39]	eval-rmse:5.78307
[40]	eval-rmse:5.78175
[41]	eval-rmse:5.77982
[42]	eval-rmse:5.77861
[43]	eval-rmse:5.7774

In [53]:
print('Model R^2 Score : ',r2_score(y_test,model.predict(testmatrix)))

Model R^2 Score :  0.4441600709108219
